In [ ]:
!pip install llama-cpp-python langgraph pyvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 21.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4127

In [ ]:
!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf

--2025-06-15 08:14:50--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.55, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/46/12/46124cd8d4788fd8e0879883abfc473f247664b987955cc98a08658f7df6b826/14466f9d658bf4a79f96c3f3f22759707c291cac4e62fea625e80c7d32169991?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.1.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.1.Q4_K_M.gguf%22%3B&Expires=1749978088&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0OTk3ODA4OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Ni8xMi80NjEyNGNkOGQ0Nzg4ZmQ4ZTA4Nzk4ODNhYmZjNDczZjI0NzY2NGI5ODc5NTVjYzk4YTA4NjU4ZjdkZjZiODI2LzE0NDY2ZjlkNjU4YmY0YTc5Zjk2YzNmM2Yy

In [ ]:
# Importing necessary library
from llama_cpp import Llama
from langgraph.graph import Graph, END
from typing import TypedDict, Annotated, Sequence
import operator
import time
from IPython.display import display, HTML, IFrame
from pyvis.network import Network
import os

In [ ]:
# ========== SETUP ==========
class AgentState(TypedDict):
    messages: Annotated[Sequence[str], operator.add]
    context: dict
    agent_logs: list  # For tracking agent interactions

llm = Llama(
    model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    n_ctx=2048,
    n_gpu_layers=40
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte

In [ ]:
# ========== AGENTS ==========
def log_interaction(agent_name: str, input: str, output: str):
    timestamp = time.strftime("%H:%M:%S", time.localtime())
    log_entry = {
        "timestamp": timestamp,
        "agent": agent_name,
        "input": input[:200] + "..." if len(input) > 200 else input,
        "output": output[:200] + "..." if len(output) > 200 else output
    }
    print(f"\n🔵 {agent_name.upper()} AGENT:\nINPUT: {log_entry['input']}\nOUTPUT: {log_entry['output']}\n")
    return log_entry

def research_agent(state: AgentState):
    query = state["messages"][-1]
    response = llm.create_chat_completion(
        messages=[{"role": "user", "content": f"""
        RESEARCH TASK: Break down this query and provide comprehensive information:
        {query}
        Include key concepts, definitions, and relevant examples."""}]
    )
    result = response['choices'][0]['message']['content']
    log_entry = log_interaction("researcher", query, result)
    return {
        "context": {"research": result},
        "agent_logs": state["agent_logs"] + [log_entry]
    }

def analysis_agent(state: AgentState):
    research = state["context"]["research"]
    response = llm.create_chat_completion(
        messages=[{"role": "user", "content": f"""
        ANALYSIS TASK: Critically examine this research:
        {research}
        Identify 3 key insights, 2 potential biases, and 1 missing perspective."""}]
    )
    result = response['choices'][0]['message']['content']
    log_entry = log_interaction("analyst", research, result)
    return {
        "context": {**state["context"], "analysis": result},
        "agent_logs": state["agent_logs"] + [log_entry]
    }

def synthesis_agent(state: AgentState):
    research = state["context"]["research"]
    analysis = state["context"]["analysis"]
    response = llm.create_chat_completion(
        messages=[{"role": "user", "content": f"""
        SYNTHESIS TASK: Combine into a polished answer:
        RESEARCH: {research}
        ANALYSIS: {analysis}
        Structure your answer with: 1) Overview 2) Key Points 3) Implications"""}]
    )
    result = response['choices'][0]['message']['content']
    log_entry = log_interaction("synthesizer", f"{research}\n{analysis}", result)
    return {
        "messages": [result],
        "agent_logs": state["agent_logs"] + [log_entry]
    }


In [ ]:

# ========== GRAPH SETUP ==========
workflow = Graph()

workflow.add_node("researcher", research_agent)
workflow.add_node("analyst", analysis_agent)
workflow.add_node("synthesizer", synthesis_agent)

workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "synthesizer")
workflow.add_edge("synthesizer", END)

workflow.set_entry_point("researcher")


In [ ]:

# ========== VISUALIZATION ==========
def visualize_workflow():
    net = Network(height="500px", width="100%", directed=True, notebook=True)

    # Add nodes
    net.add_node("researcher", label="Researcher", color="#FF6B6B", shape="box")
    net.add_node("analyst", label="Analyst", color="#4ECDC4", shape="box")
    net.add_node("synthesizer", label="Synthesizer", color="#FFE66D", shape="box")
    net.add_node("end", label="END", color="#2EC4B6", shape="ellipse")

    # Add edges
    net.add_edge("researcher", "analyst", label="research → analysis", color="#888888")
    net.add_edge("analyst", "synthesizer", label="analysis → synthesis", color="#888888")
    net.add_edge("synthesizer", "end", label="output", color="#888888")

    # Save and display
    net.show("workflow.html")
    display(IFrame(src="workflow.html", width="100%", height=500))


In [ ]:

# ========== EXECUTION ==========
print("🚀 STARTING 3-HOP MULTI-AGENT EXECUTION")
app = workflow.compile()

initial_state = {
    "messages": ["Explain quantum computing and its potential impact on cybersecurity"],
    "context": {},
    "agent_logs": []
}

# First visualize the workflow
visualize_workflow()

# Then run the agents
result = app.invoke(initial_state)

🚀 STARTING 3-HOP MULTI-AGENT EXECUTION
workflow.html


llama_perf_context_print:        load time =   11211.66 ms
llama_perf_context_print: prompt eval time =   11211.49 ms /    54 tokens (  207.62 ms per token,     4.82 tokens per second)
llama_perf_context_print:        eval time =  247834.93 ms /   392 runs   (  632.23 ms per token,     1.58 tokens per second)
llama_perf_context_print:       total time =  259366.67 ms /   446 tokens
Llama.generate: 8 prefix-match hit, remaining 434 prompt tokens to eval



🔵 RESEARCHER AGENT:
INPUT: Explain quantum computing and its potential impact on cybersecurity
OUTPUT:  Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. It has the potential to revolutionize many fiel...



llama_perf_context_print:        load time =   11211.66 ms
llama_perf_context_print: prompt eval time =   83200.43 ms /   434 tokens (  191.71 ms per token,     5.22 tokens per second)
llama_perf_context_print:        eval time =  161507.04 ms /   252 runs   (  640.90 ms per token,     1.56 tokens per second)
llama_perf_context_print:       total time =  244875.07 ms /   686 tokens
Llama.generate: 8 prefix-match hit, remaining 703 prompt tokens to eval



🔵 ANALYST AGENT:
INPUT:  Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. It has the potential to revolutionize many fiel...
OUTPUT:  3 key insights:

1. Quantum computing uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data, which can greatly speed up certain types of computation...



llama_perf_context_print:        load time =   11211.66 ms
llama_perf_context_print: prompt eval time =  135900.36 ms /   703 tokens (  193.31 ms per token,     5.17 tokens per second)
llama_perf_context_print:        eval time =  189353.28 ms /   291 runs   (  650.70 ms per token,     1.54 tokens per second)
llama_perf_context_print:       total time =  325459.26 ms /   994 tokens



🔵 SYNTHESIZER AGENT:
INPUT:  Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. It has the potential to revolutionize many fiel...
OUTPUT:  1. Overview: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. It has the potential to revolution...



In [ ]:
# ========== RESULTS ==========
print("\n🎉 FINAL OUTPUT:")
print(result["messages"][0])

print("\n📜 EXECUTION TRACE:")
for i, log in enumerate(result["agent_logs"], 1):
    print(f"\n{i}. {log['timestamp']} - {log['agent']}")
    print(f"   Input: {log['input']}")
    print(f"   Output: {log['output']}")


🎉 FINAL OUTPUT:
 1. Overview: Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. It has the potential to revolutionize many fields, including cybersecurity.
2. Key Points:
* Quantum computing uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data, which can greatly speed up certain types of computations.
* Quantum computing has the potential to greatly improve cybersecurity by allowing for the creation of more secure encryption methods, such as quantum key distribution (QKD).
* Quantum computing is still in the early stages of development, and it is not yet clear how quickly it will be able to live up to its potential.
3. Implications:
* Quantum computing has the potential to greatly improve cybersecurity by allowing for the creation of more secure encryption methods and by providing new tools for detecting and preventing cyber attacks.